In [1]:
import am4pa
from am4pa.runners import RunnerVariants
from am4pa.data_integration import DataCollector

from backend_manager import BackendManager,Commands
import os

from va4algs import Application

### Linear Algebra Problem:

In [2]:
local_problem_dir = "linnea_expressions/matrix-chain-4/"

In [3]:
operand_sizes = ["75","75","6","75","75"]

### Set up backend

In [4]:
server = "login18-1.hpc.itc.rwth-aachen.de"
uname = "as641651"
init_script = "~/.analyzer"
backend_problem_dir = "~/PhD/va4algs-exps/matrix-chain-4/"

bm = BackendManager(server=server, uname=uname)
cmds = Commands(source=init_script)

In [6]:
bm.connect()

#### Copy problem directory to backend

In [14]:
exp_exists = bm.check_if_file_exists(os.path.join(backend_problem_dir,'generate-variants-linnea.py'))
exp_exists

True

In [34]:
force = False
if not exp_exists or force:
    bm.copy_to_backend(local_problem_dir,backend_problem_dir)

scp -r linnea_expressions/matrix-chain-4/ as641651@login18-1.hpc.itc.rwth-aachen.de:~/PhD/va4algs-exps/matrix-chain-4/


perl: warning: Setting locale failed.
perl: warning: Please check that your locale settings:
	LANGUAGE = (unset),
	LC_ALL = (unset),
	LANG = "C.UTF-8"
    are supported and installed on your system.
perl: warning: Falling back to the standard locale ("C").


b''


In [36]:
assert(bm.check_if_file_exists(os.path.join(backend_problem_dir,'generate-variants-linnea.py')))

### Generate Variants

In [37]:
#cmds = Commands(source="~/.analyzer")
runner = RunnerVariants(operand_sizes, backend_problem_dir, backend_manager=bm,backend_commands=cmds)

In [38]:
generation_script = "generate-variants-linnea.py"

In [39]:
ret = runner.generate_variants_for_measurements(generation_script=generation_script)

source ~/.analyzer; cd ~/PhD/va4algs-exps/matrix-chain-4; python generate-variants-linnea.py 75 75 6 75 75 --threads=4
['New solution:.............2.02e+05\n', 'No further generation steps possible.\n', '----------------------------------\n', 'Number of nodes:                 8\n', 'Solution nodes:                  1\n', 'Data:                     1.78e+04\n', 'Best solution:            2.02e+05\n', 'Intensity:                    11.4\n', 'Number of algorithms:            6\n', 'Generated Variants.\n']
Success: Backend interactive run: Generate variants


### Collect Data

In [49]:
dc = DataCollector(os.path.join(local_problem_dir, "cluster-data"), runner.operands_dir,bm)

In [50]:
case_table = dc.get_case_table()

scp as641651@login18-1.hpc.itc.rwth-aachen.de:~/PhD/va4algs-exps/matrix-chain-4/experiments/75_75_6_75_75/case_table.csv linnea_expressions/matrix-chain-4/cluster-data


perl: warning: Setting locale failed.
perl: warning: Please check that your locale settings:
	LANGUAGE = (unset),
	LC_ALL = (unset),
	LANG = "C.UTF-8"
    are supported and installed on your system.
perl: warning: Falling back to the standard locale ("C").


b''


In [55]:
case_table.head()

,case:concept:name,case:flops,case:num_kernels
0,algorithm0,202000.0,3
1,algorithm2,979000.0,3
2,algorithm4,1760000.0,3
3,algorithm1,202000.0,3
4,algorithm5,1760000.0,3


In [56]:
event_table = dc.get_meta_table()

In [57]:
event_table.head()

,case:concept:name,concept:name,concept:flops,concept:kernel,concept:operation,timestamp:start
0,algorithm0,gemm_6.75e+04,67500.0,"gemm!('N', 'N', 1.0, ml2, ml3, 0.0, ml4)",tmp3 = (C D),2022-10-18 16:33:15.634146
1,algorithm0,gemm_6.75e+04,67500.0,"gemm!('N', 'N', 1.0, ml0, ml1, 0.0, ml5)",tmp1 = (A B),2022-10-18 16:33:16.634146
2,algorithm0,gemm_6.75e+04,67500.0,"gemm!('N', 'N', 1.0, ml5, ml4, 0.0, ml6)",tmp6 = (tmp1 tmp3),2022-10-18 16:33:17.634146
3,algorithm2,gemm_6.75e+04,67500.0,"gemm!('N', 'N', 1.0, ml0, ml1, 0.0, ml4)",tmp1 = (A B),2022-10-18 16:36:35.634146
4,algorithm2,gemm_6.75e+04,67500.0,"gemm!('N', 'N', 1.0, ml4, ml2, 0.0, ml5)",tmp4 = (tmp1 C),2022-10-18 16:36:36.634146


### Initialize va4algs

In [5]:
flask_port = 5003

app = Application(local_problem_dir=local_problem_dir,
                 flask_port=flask_port,
                 default_operand_size=operand_sizes,
                 language="linnea",
                 app_dir="app/")

app.use_measurement_server(server=server,
                          uname=uname,
                          problem_dir=backend_problem_dir,
                          init_script=init_script)

app.initialize()

### Run a local Flask server

In [6]:
!python app/app.py

 * Serving Flask app "app" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
 * Running on http://127.0.0.1:5003/ (Press CTRL+C to quit)
 * Restarting with watchdog (inotify)
 * Debugger is active!
 * Debugger PIN: 650-530-613
127.0.0.1 - - [19/Oct/2022 15:45:08] "GET /hello HTTP/1.1" 200 -
^C


<a class="btn btn-primary btn-lg" href="app/index.html" role="button">Analyse</a>

In [2]:
int